# Project 1 - Team Tasty



In [ ]:
pip install kaggle --upgrade

In [84]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
from scipy.stats import linregress
import numpy as np

# Study data files
fastfood_path = "Resources/Datafiniti_Fast_Food_Restaurants_May19.csv"

# Read the mouse data and the study results
fastfood_data = pd.read_csv(fastfood_path)

fastfood_data.head()

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,name,postalCode,province,sourceURLs,websites
0,AWrSh_KgsVYjT2BJAzaH,2019-05-19T23:58:05Z,2019-05-19T23:58:05Z,2555 11th Avenue,"Fast Food Restaurants,Hamburgers and Hot Dogs,...",Accommodation & Food Services,Greeley,US,us/co/greeley/255511thavenue/554191587,40.39629,-104.69699,Carl's Jr.,80631,CO,https://www.yellowpages.com/greeley-co/mip/car...,https://www.carlsjr.com/?utm_source=Yextandutm...
1,AWEKlA-LIxWefVJwxG9B,2018-01-18T18:30:23Z,2019-05-19T23:45:05Z,2513 Highway 6 And 50,"Restaurant,Mexican Restaurants,Fast Food Resta...",Accommodation & Food Services,Grand Junction,US,us/co/grandjunction/2513highway6and50/1550891556,39.08135,-108.58689,Del Taco,81505,CO,http://www.citysearch.com/profile/772076870/gr...,http://www.deltaco.com
2,AWrSfAcYsVYjT2BJAzPt,2019-05-19T23:45:04Z,2019-05-19T23:45:04Z,1125 Patterson Road,"Sandwich Shops,Fast Food Restaurants,Restauran...",Accommodation & Food Services,Grand Junction,US,us/co/grandjunction/1125pattersonroad/-2137447852,39.09148,-108.55411,Which Wich,81506,CO,https://www.yellowpages.com/grand-junction-co/...,http://www.whichwich.com
3,AWrSa3NAQTFama1Xpkbz,2019-05-19T23:26:58Z,2019-05-19T23:26:58Z,3455 N Salida Court,"Fast Food Restaurants,Mexican Restaurants,Rest...",Accommodation & Food Services,Aurora,US,us/co/aurora/3455nsalidacourt/1143321601,39.76369,-104.77671,Chipotle Mexican Grill,80011,CO,https://www.yellowpages.com/aurora-co/mip/chip...,http://www.chipotle.com
4,AWrSaVGzZ4Yw-wtdgcaB,2019-05-19T23:24:38Z,2019-05-19T23:24:38Z,5225 E Colfax Avenue,"Fast Food Restaurants,Mexican Restaurants,Rest...",Accommodation & Food Services,Denver,US,us/co/denver/5225ecolfaxavenue/-864103396,39.74044,-104.92636,Taco Bell,80220,CO,https://www.yellowpages.com/denver-co/mip/taco...,https://locations.tacobell.com/co/denver/5225-...


In [85]:
fastfood_data["name"].value_counts()

McDonald's             765
Subway                 667
SUBWAY                 543
Taco Bell              529
Pizza Hut              474
                      ... 
Lok Yaun Restaurant      1
EVOS South Tampa         1
To Go's                  1
The Greenery Cafe        1
Classic Pizza            1
Name: name, Length: 1545, dtype: int64

We use value count to filter out restaurants that only have one location since having only one would mean that disqualify it from an analysis of US fast food restaurant.

In [86]:
# Combining Similarly-named Restaurants together


reduced_fastfood_df = fastfood_data.replace({# Subway Restaurants
                                            "Subway Sandwiches Salads": "Subway",
                                            "Subway Meadows Park SC":"Subway",
                                            "SUBWAY Restaurant":"Subway",
                                            "Moz Subway":"Subway",
                                            "Subway Sandwiches":"Subway",
                                            "Subway 16 Mile":"Subway",
                                            "Desert Subway":"Subway",
                                            "SUBWAY":"Subway",
                                             # McDonald
                                            "Mcdonalds":"McDonald's",
                                            "McDonald's - CLOSED":"McDonald's",
                                            "McDonalds Family Restaurant":"McDonald's",
                                            "Mc Donald's":"McDonald's",
                                            # Carl's Jr. + Hardee's
                                            "Carl's Jr":"Carl's Jr.",
                                            "Carls Jr":"Carl's Jr.",
                                            "Carls Jr Green Burrito":"Carl's Jr.",
                                            "Hardee's":"Carl's Jr.",
                                            "Hardees":"Carl's Jr.",
                                            "Hardees Red Burrito":"Carl's Jr.",
                                            "Hardee's/red Burrito":"Carl's Jr.",
                                            "Hardee's - Loris":"Carl's Jr",
                                            "Hardee's Restaurant":"Carl's Jr",
                                            "Hardee's Restaurants":"Carl's Jr",
                                            # KFC
                                            "Kentucky Fried Chicken": "Kfc",
                                            "KFC - Kentucky Fried Chicken":"Kfc",
                                            "KFC Kentucky Fried Chicken":"Kfc",
                                            "KFC - Closed":"Kfc",
                                            "KFC AW":"Kfc",
                                            # Burger King
                                            "Burger King - Temporarily Closed":"Burger King",
                                            "Burger KingÂ®":"Burger King",
                                            "Burger King - Closed":"Burger King",
                                            "Burger KingPopeyes":"Burger King",
                                            # Sonic Drive-In
                                            "SONIC Drive In":"Sonic Drive-In",
                                            "Sonic DriveIn":"Sonic Drive-In",
                                            "Sonic Drive-In - Temporarily Closed":"Sonic Drive-In",
                                            "Sonic":"Sonic Drive-In",
                                            "Sonic Drive-In - Closed":"Sonic Drive-In",
                                            # Dairy Queen
                                            "Dairy Queen Grill Chill":"Dairy Queen",
                                            "Dairy Queen Grill & Chill":"Dairy Queen",
                                            "Dairy Queen (Treat)":"Dairy Queen",
                                            "Dairy Queen-Russo":"Dairy Queen",
                                            "Dairy Queen - Closed":"Dairy Queen",
                                            "Dairy Queen (Treat) - Seasonally Closed":"Dairy Queen",
                                            "Dairy Queen Grill and Chill":"Dairy Queen",
                                            "Dairy Queen Grill Chill - Temporarily Closed":"Dairy Queen",
                                            "Dairy Queen Orange Julius":"Dairy Queen",
                                            # Wendy's
                                            "Wendys":"Wendy's",
                                            "Wendy's - Temporarily Closed":"Wendy's",
                                            # Domino's Pizza
                                            "Dominos Pizza":"Domino's Pizza",
                                            # Arby's
                                            "Arbys":"Arby's",
                                            "Arbys CLOSED":"Arby's",
                                            "Arby's - Closed":"Arby's - Coming Soon",
                                            "Arby's - Coming Soon":"Arby's",
                                            # Chick-fil-A
                                            "ChickfilA":"Chick-fil-A",
                                            "Chick-Fil-A Of Randolph Mall":"Chick-fil-A",
                                            "Truett's Chick-fil-A":"Chick-fil-A",
                                            # Chipotle Mexican Grill
                                            "Chipotle":"Chipotle Mexican Grill",
                                            # Jimmy John's
                                            "Jimmy Johns":"Jimmy John's",
                                            "Jimmy John's Boyton Beach":"Jimmy John's",
                                            # Panda Express
                                            "Panda Express - Inside Vons + Inside Vons +":"Panda Express",
                                            # Steak 'n Shake
                                            "Steak N Shake":"Steak 'n Shake",
                                            # Church's Chicken
                                            "Churchs Chicken":"Church's Chicken",
                                            "Church's Chicken of Birmingham":"Church's Chicken",
                                            # Quiznos
                                            "Quiznos Sub":"Quiznos",
                                            "Quiznos Closed":"Quiznos",
                                            # Popeyes Louisiana Kitchen
                                            "Popeyes Louisiana Kitchen":"Popeye's Louisiana Kitchen",
                                            "Popeyes Chicken Biscuits":"Popeye's Louisiana Kitchen",
                                            "Popeyes Chicken and Biscuits":"Popeye's Louisiana Kitchen",
                                            "Popeye's Chicken & Biscuits":"Popeye's Louisiana Kitchen",
                                            "Popeyes":"Popeye's Louisiana Kitchen",
                                            # Five Guys Burgers Fries
                                            "Five Guys":"Five Guys Burgers Fries",
                                            "Five Guys Burgers & Fries":"Five Guys Burgers Fries",
                                            # Checkers
                                            "Checkers Drive-In Rstrnt Inc":"Checkers",
                                            "Checker's Drive In Restaurant":"Checkers",
                                            # Whataburger
                                            "Whatburger of East Texas Corporate":"Whataburger",
                                            "Whataburger of East Texas":"Whataburger",
                                            "Whataburger - Closed":"Whataburger",
                                            # Culver's
                                            "Culvers":"Culver's",
                                            "Long John Silvers":"Long John Silver's",
                                            "Bojangles Famous Chicken n Biscuits":"Bojangles' Famous Chicken 'n Biscuits",
                                            "Bojangles":"Bojangles' Famous Chicken 'n Biscuits",
                                            "Jack's Family Restaurants":"Jack's",
                                            "Jack's Old Fashion Hamburger":"Jack's",
                                            "Jack's Hamburgers":"Jack's",
                                            "Jacks":"Jack's",
                                            "Bob Evans Restaurant":"Bob Evans",
                                            "Dunkin' Donuts":"Dunkin Donuts",
                                            "Dunkin Donuts/baskinrobbins":"Dunkin Donuts",
                                            "Dunkin'":"Dunkin Donuts",
                                            "AW All-American Food":"AW Restaurants",
                                            "AW":"AW Restaurants",
                                            "AWRestaurants":"AW Restaurants",
                                            "Awful Annie's":"AW Restaurants",
                                            "AW Restaurant":"AW Restaurants",
                                            "InNOut Burger":"In-N-Out Burger",
                                            "Rallys":"Rally's",
                                            "Rally's Hamburgers":"Rally's",
                                            "Rallys Hamburgers":"Rally's",
                                            "Papa John's Pizza":"Papa Johns Pizza",
                                            "Charleys Grilled Subs":"Charley's Grilled Subs"})


reduced_fastfood_df["name"].value_counts()

Subway                            1219
McDonald's                         784
Taco Bell                          529
Pizza Hut                          474
Burger King                        453
                                  ... 
Leone's Italian Restaurants          1
Casola's Pizzeria & Sub Shop         1
Chow To Go Chinese Restaurant        1
Syros Greek Italian Restaurant       1
Miyoko Sushi                         1
Name: name, Length: 1452, dtype: int64

In [87]:
# Create Reduced dataframe to contain only the required 
fields = ["name","categories","address","city","postalCode","province","country","latitude","longitude"]

In [88]:
reduced_fastfood_df = reduced_fastfood_df[fields]
reduced_fastfood_df = reduced_fastfood_df.rename(columns={'province':'state'})

In [89]:
simple_fastfood = reduced_fastfood_df[["name","state"]]
grouped_series=simple_fastfood.groupby("name").size().sort_values(ascending=False)
grouped_df = grouped_series.to_frame()
grouped_df

,0
name,
Subway,1219
McDonald's,784
Taco Bell,529
Pizza Hut,474
Burger King,453
...,...
Mr Submarine,1
Mr. Hero,1
Mrs Fields,1


In [93]:
counts_fastfood = pd.merge(reduced_fastfood_df,grouped_df,how = "left", on = "name")
counts_fastfood = counts_fastfood.rename(columns = {0:"count"})
counts_fastfood.head()

,categories,address,city,postalCode,state,country,latitude,longitude,count
name,,,,,,,,,
Carl's Jr.,"Fast Food Restaurants,Hamburgers and Hot Dogs,...",2555 11th Avenue,Greeley,80631,CO,US,40.39629,-104.69699,205
Del Taco,"Restaurant,Mexican Restaurants,Fast Food Resta...",2513 Highway 6 And 50,Grand Junction,81505,CO,US,39.08135,-108.58689,30
Which Wich,"Sandwich Shops,Fast Food Restaurants,Restauran...",1125 Patterson Road,Grand Junction,81506,CO,US,39.09148,-108.55411,7
Chipotle Mexican Grill,"Fast Food Restaurants,Mexican Restaurants,Rest...",3455 N Salida Court,Aurora,80011,CO,US,39.76369,-104.77671,142
Taco Bell,"Fast Food Restaurants,Mexican Restaurants,Rest...",5225 E Colfax Avenue,Denver,80220,CO,US,39.74044,-104.92636,529


In [92]:
counts_fastfood.count()

categories    10000
address       10000
city          10000
postalCode    10000
state         10000
country       10000
latitude      10000
longitude     10000
count         10000
dtype: int64

In [94]:
reduced_fastfood_df = reduced_fastfood_df.set_index("name")
reduced_fastfood_df.head()

KeyError: "None of ['name'] are in the columns"

In [95]:
reduced_fastfood_df.count()

categories    10000
address       10000
city          10000
postalCode    10000
state         10000
country       10000
latitude      10000
longitude     10000
dtype: int64

In [97]:
# Qualified filters out restaurants with less than 10 locations only from analysis
minimum_location = 10
filter_fastfood = counts_fastfood.loc[(counts_fastfood["count"]>minimum_location)]
filter_fastfood = filter_fastfood.reset_index()
filter_fastfood

,name,categories,address,city,postalCode,state,country,latitude,longitude,count
0,Carl's Jr.,"Fast Food Restaurants,Hamburgers and Hot Dogs,...",2555 11th Avenue,Greeley,80631,CO,US,40.396290,-104.696990,205
1,Del Taco,"Restaurant,Mexican Restaurants,Fast Food Resta...",2513 Highway 6 And 50,Grand Junction,81505,CO,US,39.081350,-108.586890,30
2,Chipotle Mexican Grill,"Fast Food Restaurants,Mexican Restaurants,Rest...",3455 N Salida Court,Aurora,80011,CO,US,39.763690,-104.776710,142
3,Taco Bell,"Fast Food Restaurants,Mexican Restaurants,Rest...",5225 E Colfax Avenue,Denver,80220,CO,US,39.740440,-104.926360,529
4,McDonald's,"Fast Food Restaurants,Hamburgers and Hot Dogs,...",1350 W Colfax Avenue,Denver,80204,CO,US,39.739670,-105.004670,784
...,...,...,...,...,...,...,...,...,...,...
8240,Subway,"Carry-out food,Delicatessens,Take Out Restaura...",202 W Camp Wisdom Rd,Duncanville,75116,TX,US,32.661820,-96.908570,1219
8241,Bob Evans,"American Restaurants,Carry-out food,Restaurant...",121 Marlene Dr,Nicholasville,40356,KY,US,37.954950,-84.537840,40
8242,McDonald's,"Carry-out food,Hot dog restaurants,Fast Food R...",2601 Plover Rd,Plover,54467,WI,US,44.453180,-89.543170,784
8243,Pizza Hut,"Carry-out food,Caterers,Restaurant delivery se...",170 E Main St,Hendersonville,37075,TN,US,36.308262,-86.607339,474
